In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from seapopym.configuration.no_transport import ForcingParameter, ForcingUnit

from seapopym_optimization.constraint.energy_transfert_constraint import EnergyCoefficientConstraint
from seapopym_optimization.cost_function import SimpleRootMeanSquareErrorCostFunction, TimeSeriesObservation
from seapopym_optimization.functional_group import NoTransportFunctionalGroup, Parameter
from seapopym_optimization.functional_group.base_functional_group import FunctionalGroupSet
from seapopym_optimization.genetic_algorithm.simple_genetic_algorithm import (
    SimpleGeneticAlgorithm,
    SimpleGeneticAlgorithmParameters,
)
from seapopym_optimization.model_generator import NoTransportModelGenerator


/Users/adm-lehodey/Documents/Workspace/Projects/Seapopym-optimisation/.venv/lib/python3.13/site-packages/seapopym/standard/units.py:9: UserWarning: Import(s) unavailable to set up matplotlib support...skipping this portion of the setup.
  import cf_xarray.units  # noqa: F401


In [2]:
fg1 = NoTransportFunctionalGroup(
    name="FG1",
    day_layer=0,
    night_layer=0,
    energy_transfert=Parameter(name="FG1_energy_transfert", lower_bound=0, upper_bound=1),
    lambda_temperature_0=Parameter(name="lambda_temperature_0", lower_bound=0, upper_bound=100),
    gamma_lambda_temperature=Parameter(name="gamma_lambda_temperature", lower_bound=0, upper_bound=1),
    tr_0=Parameter(name="tr_0", lower_bound=0, upper_bound=10),
    gamma_tr=Parameter(name="gamma_tr", lower_bound=-1, upper_bound=0),
)

fg2 = NoTransportFunctionalGroup(
    name="FG2",
    day_layer=1,
    night_layer=1,
    energy_transfert=Parameter(name="FG2_energy_transfert", lower_bound=0, upper_bound=1),
    lambda_temperature_0=Parameter(name="lambda_temperature_0", lower_bound=0, upper_bound=100),
    gamma_lambda_temperature=Parameter(name="gamma_lambda_temperature", lower_bound=0, upper_bound=1),
    tr_0=Parameter(name="tr_0", lower_bound=0, upper_bound=10),
    gamma_tr=Parameter(name="gamma_tr", lower_bound=-1, upper_bound=0),
)

In [3]:
constraint = EnergyCoefficientConstraint(
    parameters_name=["FG1_energy_transfert", "FG2_energy_transfert"],
    min_energy_coef_value=0,
    max_energy_coef_value=1,
)

In [4]:
fg_set = FunctionalGroupSet(
    functional_groups=[fg1, fg2],
)

In [5]:
fg_set.unique_functional_groups_parameters_ordered().keys()

dict_keys(['FG1_energy_transfert', 'lambda_temperature_0', 'gamma_lambda_temperature', 'tr_0', 'gamma_tr', 'FG2_energy_transfert'])

In [6]:
parameters_set = [1, np.inf, 1, 1, -1, 1]
functional_group_parameters = fg_set.generate(parameters_set)
functional_group_parameters

[{'day_layer': 0,
  'night_layer': 0,
  'energy_transfert': 1,
  'lambda_temperature_0': inf,
  'gamma_lambda_temperature': 1,
  'tr_0': 1,
  'gamma_tr': -1},
 {'day_layer': 1,
  'night_layer': 1,
  'energy_transfert': 1,
  'lambda_temperature_0': inf,
  'gamma_lambda_temperature': 1,
  'tr_0': 1,
  'gamma_tr': -1}]

In [7]:
temperature = xr.DataArray(
    data=np.zeros((5, 1, 1, 2)),
    dims=["time", "latitude", "longitude", "depth"],
    coords={
        "time": pd.date_range("2023-01-01", periods=5, freq="D"),
        "latitude": [0],
        "longitude": [0],
        "depth": [0, 1],
    },
    name="temperature",
    attrs={
        "units": "Celsius",
        "long_name": "Sea surface temperature",
        "standard_name": "sea_surface_temperature",
    },
)
primary_production = xr.DataArray(
    data=np.ones((5, 1, 1)),
    dims=["time", "latitude", "longitude"],
    coords={
        "time": pd.date_range("2023-01-01", periods=5, freq="D"),
        "latitude": [0],
        "longitude": [0],
    },
    name="primary_production",
    attrs={
        "units": "kg/m^2/day",
        "long_name": "Primary production",
        "standard_name": "primary_production",
    },
)
observation = xr.DataArray(
    data=np.ones((5, 1, 1, 2)),
    dims=["time", "latitude", "longitude", "layer"],
    coords={
        "time": pd.date_range("2023-01-01", periods=5, freq="D"),
        "latitude": [0],
        "longitude": [0],
        "layer": [0, 1],
    },
    attrs={
        "units": "kg/m^2",
    },
)
observation.time.attrs = {"axis": "T"}
temperature.time.attrs = {"axis": "T"}
primary_production.time.attrs = {"axis": "T"}

observation.latitude.attrs = {"axis": "Y"}
temperature.latitude.attrs = {"axis": "Y"}
primary_production.latitude.attrs = {"axis": "Y"}

observation.longitude.attrs = {"axis": "X"}
temperature.longitude.attrs = {"axis": "X"}
primary_production.longitude.attrs = {"axis": "X"}

observation.layer.attrs = {"axis": "Z"}
temperature.depth.attrs = {"axis": "Z"}

forcing_parameter = ForcingParameter(
    temperature=ForcingUnit(forcing=temperature),
    primary_production=ForcingUnit(forcing=primary_production),
)
with xr.set_options(keep_attrs=True):
    observation_2 = observation.sel(layer=[1]) / 2

In [8]:
observation_1 = TimeSeriesObservation(
    name="Observation_all",
    observation=observation,
)
observation_2 = TimeSeriesObservation(
    name="Observation_meso",
    observation=observation_2,
)

In [9]:
model_generator = NoTransportModelGenerator(forcing_parameters=forcing_parameter)

In [10]:
model = model_generator.generate(functional_group_parameters=functional_group_parameters)

In [11]:
model.run()

In [12]:
model.state

<xarray.Dataset> Size: 936B
Dimensions:                       (time: 5, latitude: 1, longitude: 1,
                                   functional_group: 2, cohort: 1, depth: 2)
Coordinates:
  * time                          (time) datetime64[ns] 40B 2023-01-01 ... 20...
  * latitude                      (latitude) int64 8B 0
  * longitude                     (longitude) int64 8B 0
  * functional_group              (functional_group) int64 16B 0 1
  * cohort                        (cohort) int64 8B 0
  * depth                         (depth) int64 16B 0 1
Data variables: (12/28)
    biomass                       (functional_group, time, latitude, longitude) float64 80B ...
    recruited                     (functional_group, time, latitude, longitude) float64 80B ...
    mortality_field               (functional_group, time, latitude, longitude) float64 80B ...
    mask_temperature              (functional_group, time, latitude, longitude, cohort) bool 10B ...
    min_temperature               (functional_group, cohort) float64 16B -0.0...
    primary_production_by_fgroup  (functional_group, time, latitude, longitude) float64 80B ...
    ...                            ...
    max_timestep                  (functional_group, cohort) int64 16B 1 1
    mean_timestep                 (functional_group, cohort) float64 16B 1.0 1.0
    timestep                      int64 8B 1
    angle_horizon_sun             float64 8B 0.0
    compute_initial_conditions    bool 1B False
    compute_preproduction         bool 1B False

In [13]:
cost_function = SimpleRootMeanSquareErrorCostFunction(
    model_generator=model_generator,
    observations=[observation_1, observation_2],
    functional_groups=[fg1, fg2],
    root_mse=True,
    centered_mse=False,
    normalized_mse=False,
)
partial_cost_function = cost_function.generate()
partial_cost_function(parameters_set)


(np.float64(0.0), np.float64(0.5))

In [14]:
metaparam = SimpleGeneticAlgorithmParameters(
    ETA=4, INDPB=0.2, CXPB=0.5, MUTPB=0.2, NGEN=10, POP_SIZE=10, cost_function_weight=(-1, -1)
)
genetic_algorithm = SimpleGeneticAlgorithm(
    meta_parameter=metaparam,
    cost_function=cost_function,
    constraint=[constraint],
)


In [15]:
genetic_algorithm.optimize()

,mean,std,min,max,valid,from_previous_generation
generation,,,,,,
0,-1.245420,0.196988,-1.500000,-1.042332,4,0.00
1,-1.318169,0.178839,-1.500000,-1.065369,8,0.25
2,-1.251254,0.167409,-1.500000,-1.065369,10,0.30
3,-1.168062,0.168289,-1.500000,-0.936018,10,0.40
4,-1.071739,0.086151,-1.288268,-0.936018,10,0.40
5,-1.069742,0.077514,-1.249447,-0.922674,10,0.20
6,-1.051830,0.045584,-1.078118,-0.922674,10,0.60
7,-1.035740,0.059596,-1.065369,-0.922674,10,0.80
8,-1.016812,0.072001,-1.112568,-0.922674,10,0.20


Generations: 100%|██████████| 10/10 [00:09<00:00,  1.10it/s]


AttributeError: 'SimpleRootMeanSquareErrorCostFunction' object has no attribute 'forcing_parameters'